In [1]:
import pygame
import configparser
import pygame.sprite
import random
import pygame
import time
import sys 

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


NOTES: 
-an event is anything that happens inside your game window

pygame notes:

use pygame.image.load("image source here").convert() to get an image you want into your environment. once this is in your environment, which should be in the initialization area of our classes, if need be, call it in another function of your class if youre creating an image somewhere else, though for basic stuff like NPCs and players it should just be the image in our initialization...


.rect NOTES FROM ONLINE TO REMEMBER:

The first two arguments get assigned to the x and y attributes of the rect, the coordinates of the topleft corner. So if you want to create a rect that is positioned at the coordinates (200, 300), you can write: rect = pygame.Rect(200, 300, 40, 50). The third and fourth argument are indeed the dimensions of the resulting rect instance.


n Pygame, the .rect attribute is a crucial part of working with rectangular areas. Let’s dive into what it represents:

Rect Objects:
Pygame uses Rect objects to store and manipulate rectangular areas.
A Rect can be created using a combination of the following values:
left: The x-coordinate of the top-left corner.
top: The y-coordinate of the top-left corner.
width: The width of the rectangle.
height: The height of the rectangle.


Common Uses:
Rect objects are used for:
Storing the blit position of sprites or images.
Collision detection (e.g., with colliderect or collidepoint methods).
Remember, the first two arguments of a Rect represent the x and y coordinates of the top-left corner, allowing you to position the rectangle precisely on the screen1.

What to finish:
-get character backgrounds transparent, fix dialogue
-finish start screen and combine with main gui code
-add music with API
-find out what other API's we can use to incorporate what we learned in class besides useful info from tkinter
-figure out how to get an image to pop up with text when you find the right item on a character, then have it bring you back to start screen or give an option to go back to start screen (IF WE HAVE TIME.)

In [6]:
#VARIABLES

TILESIZE = 13
PC_INIT_X = 600
PC_INIT_Y = 600
PLAYER_SPEED = 5
NPC_INIT_X = 300
NPC_INIT_Y = 600
NPC_SPEED = 3
PLAYER_LAYER = 2
NPC_LAYER = 1
WALL_LAYER = 3

NPC2_SPEED = 3
NPC2_LAYER = 3
NPC2_INIT_X = 500
NPC2_INIT_Y = 400

#WALLS ARE PARAMETERIZED BY X,Y, width height
#coordinates on pygame starts in top left corner as (0,0), down axis is y in positive and side axis is x and positive


#FIGURE OUT WHERE TO PUT CODE FOR INTRO SCREEN WHEN CELESTE FINISHES
#CLASS FOR ENTIRE GAME, EVERY OTHER CLASS/FUNCTION WORKING WITH THIS!!!! 
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption('rpg test')
        self.screen = pygame.display.set_mode((1067, 779))
        self.clock = pygame.time.Clock()
        self.all_sprites = pygame.sprite.LayeredUpdates()
        self.wallup = Wall(0, 0, 3, 979)
        self.wallleft = Wall(0, 0, 1340, 50)
        self.wallright = Wall(1067, 0, 50, 900)
        self.walldown = Wall(0, 779, 2000, 500) 
        self.wallupleft = Wall(0, 0, 300, 300)
        self.wallupright = Wall(760, 30, 800, 290) #(MOVES RIGHTWARDS, MOVES DOWNWARDS, WIDTH, HEIGHT)
        self.player = Player(self)
        self.NPC = NPC(self)
        self.NPC2 = NPC2(self)
        
        #LOADS BACKGROUND IMAGE:
        self.background = pygame.image.load("C:\\Users\\chris\\OneDrive\\Documents\\HARP_151_PROJ_PYGAME\\spaceship_interior_small.png")
        self.text_box = pygame.image.load("C:\\Users\\chris\\OneDrive\\Documents\\HARP_151_PROJ_PYGAME\\text_box.png")
        self.clicked_text = None #syntax to run text on gui
    
    #CREATES EVENT FOR MOUSE CLICKS ON OUR NPCS
    def event(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # clicks left side of mouse button 
                    mouse_position = pygame.mouse.get_pos()
                    if self.NPC2.rect.collidepoint(mouse_position):
                        self.clicked_text = "You found the item!!!!"
                        #FIGURE OUT HOW TO EXIT AFTER THIS AND PUT A CONGRATS IMAGE...
                        
                    elif self.NPC.rect.collidepoint(mouse_position):
                        self.clicked_text = "This character doesn't have the item, they dont like that you dont trust them..."



    #DISPLAYS TEXT FROM MOUSE CLICKS           
    def display_text(self, text, color, position, bg_color):
        font = pygame.font.SysFont("poorrichard", 30)
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect()
        text_rect.topleft = position
        
        if bg_color: #creates rectangle to put text into
            pygame.draw.rect(self.screen, bg_color, (text_rect.left - 30, text_rect.top - 5, text_rect.width + 50, text_rect.height + 20))
        
        self.screen.blit(text_surface, text_rect) #BLITS TEXT ONTO SCREEN
        


#updates sprites
    def update(self):
        self.all_sprites.update()

#updates background, we need this so player and sprites dont create a trail behind them
    def show_change(self):
        self.screen.blit(self.background, (0, 0)) #blit is like .pack in tkinter, we're blitting background image
        self.all_sprites.draw(self.screen) #draws sprite onto screen
        if self.clicked_text:
            self.display_text(self.clicked_text, "white", (100, 100), (83, 125, 154))  # Display text with black background
        pygame.display.update()

    def run(self): #RUNS GAME
        while True:
            self.event()
            self.update()
            self.show_change()
            self.clock.tick(60)

#movement, wall collision and image for player
class Player(pygame.sprite.Sprite):
    def __init__(self, game):
        super().__init__()
        self.game = game
        self.layer = PLAYER_LAYER
        self.image = pygame.image.load("C:\\Users\\chris\\OneDrive\\Documents\\Python_Scripts\\pixel_guy.png").convert()
        self.rect = self.image.get_rect(x=PC_INIT_X, y=PC_INIT_Y)
        self.game.all_sprites.add(self)
        self.speed = PLAYER_SPEED

    def move_player(self):
        keys = pygame.key.get_pressed()
        self.movex = (keys[pygame.K_RIGHT] - keys[pygame.K_LEFT]) * self.speed
        self.movey = (keys[pygame.K_DOWN] - keys[pygame.K_UP]) * self.speed

        self.rect.x += self.movex
        self.rect.y += self.movey

        if self.rect.colliderect(self.game.wallup.rect) or self.rect.colliderect(self.game.wallleft.rect) or self.rect.colliderect(self.game.wallupleft.rect) or self.rect.colliderect(self.game.wallupright.rect) or self.rect.colliderect(self.game.wallright.rect) or self.rect.colliderect(self.game.walldown.rect):
            self.rect.x -= self.movex
            self.rect.y -= self.movey

    def update(self):
        self.move_player()

#movements and image for NPC1
class NPC(pygame.sprite.Sprite):
    def __init__(self, game):
        super().__init__()
        self.game = game
        self.layer = NPC_LAYER
        self.image = pygame.image.load("C:\\Users\\chris\\OneDrive\\Documents\\HARP_151_PROJ_PYGAME\\small_spaceman_clear.png").convert()
        self.rect = self.image.get_rect(x=NPC_INIT_X, y=NPC_INIT_Y)
        self.game.all_sprites.add(self)
        self.speed = NPC_SPEED

    def move_NPC(self):
        movement_options = [0, TILESIZE, -TILESIZE]
        movement_weight = [99, 1, 1]

        #parameterizes movement in x and y, then below we use the premethods of .rect.x and .rect.y of npc and .movex
        #.movey to set npc movement 
        self.movex = random.choices(movement_options, movement_weight)[0] * self.speed
        self.movey = random.choices(movement_options, movement_weight)[0] * self.speed

        self.rect.x += self.movex
        self.rect.y += self.movey
        
        #should block npc from leaving gui but it catches them instead, need to fix npc movements to solve issue
        #if self.rect.colliderect(self.game.wallup.rect) or self.rect.colliderect(self.game.wallleft.rect) or self.rect.colliderect(self.game.wallupleft.rect) or self.rect.colliderect(self.game.wallupright.rect) or self.rect.colliderect(self.game.wallright.rect) or self.rect.colliderect(self.game.walldown.rect):
            #self.rect.x -= self.movex
            #self.rect.y -= self.movey
            
    def update(self):
        self.move_NPC()
        
#movements and image for NPC2
class NPC2(pygame.sprite.Sprite):
    def __init__(self, game):
        super().__init__()
        self.game = game
        self.layer = NPC2_LAYER
        self.image = pygame.image.load("C:\\Users\\chris\\OneDrive\\Documents\\HARP_151_PROJ_PYGAME\\small_spaceman_clear.png").convert()
        self.rect = self.image.get_rect(x=NPC2_INIT_X, y=NPC2_INIT_Y)
        self.game.all_sprites.add(self)
        self.speed = NPC2_SPEED

    def move_NPC2(self):
        movement_options = [0, TILESIZE, -TILESIZE]
        movement_weight = [50, 0.5, 0.5]

        self.movex = random.choices(movement_options, movement_weight)[0] * self.speed #initializes movement in x
        self.movey = random.choices(movement_options, movement_weight)[0] * self.speed #initializes movement in y
        
        #should block npc from leaving map, sort of catches them instead, need to fix npc movement to work properly
        #if self.rect.colliderect(self.game.wallup.rect) or self.rect.colliderect(self.game.wallleft.rect) or self.rect.colliderect(self.game.wallupleft.rect) or self.rect.colliderect(self.game.wallupright.rect) or self.rect.colliderect(self.game.wallright.rect) or self.rect.colliderect(self.game.walldown.rect):
            #self.rect.x -= self.movex
            #self.rect.y -= self.movey
            
        self.rect.x += self.movex #moves npc in x
        self.rect.y += self.movey #moves npc in y

    def update(self):
        self.move_NPC2()

#CREATES WALL AKA BOUNDS THAT DONT LET PEOPLE LEAVE REGION
class Wall(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height):
        super().__init__()
        self.image = pygame.Surface((width, height))
        self.image.fill((255, 255, 255))
        self.rect = self.image.get_rect(topleft=(x, y))


Game().run()

#WORK CITED:
#General syntax: https://www.pygame.org/docs/ref/font.html/
#font: https://stackoverflow.com/questions/38001898/what-fonts-can-i-use-with-pygame-font-font
#loading images into sprite/syntax help: https://stackoverflow.com/questions/47082209/pygame-loading-images-in-sprites
#understanding .rect method: https://stackoverflow.com/questions/47829248/pygame-what-is-the-rect-arguments-exactly-and-what-do-they-represent
#how to detect mouse click: https://stackoverflow.com/questions/10990137/pygame-mouse-clicking-detection
#PYGAME DOCUMENTATION: https://www.pygame.org/docs/

SystemExit: 

In [ ]:
pygame.font.get_fonts()